In [150]:
from discovery_utils.getters import hansard
from datetime import datetime, timedelta

In [168]:
Hansard = hansard.HansardGetter()
debates_df = Hansard.get_debates_parquet()
debates_df.date.max()

2024-11-08 16:33:43,195 - discovery_utils.getters.hansard - INFO - Downloading debates parquet file: data/policy_scanning_data/enriched/HansardDebates.parquet


'2024-11-06'

In [166]:
len(debates_df)

1572718

In [170]:
debates_df.date.max()

'2024-11-06'

In [146]:
people_df = Hansard.get_people_metadata()

2024-11-08 16:24:17,684 - discovery_utils.getters.hansard - INFO - Downloading people metadata
2024-11-08 16:24:31,792 - discovery_utils.getters.hansard - INFO - Successfully downloaded and saved people metadata


In [171]:
labelstore_df = Hansard.get_labelstore()

2024-11-08 16:35:07,180 - discovery_utils.getters.hansard - INFO - Attempting to download label store: data/policy_scanning_data/enriched/HansardDebates_LabelStore_keywords.csv


In [172]:
len(labelstore_df)

529168

In [167]:
len(labelstore_df)

529168

In [161]:
# end_date = "2024-11-08"
end_date = "2024-10-10"
# Convert the string to a datetime object
end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
# Subtract one week (7 days)
one_week_ago = end_date_dt - timedelta(weeks=1)
# Convert back to string format if needed
start_date = one_week_ago.strftime("%Y-%m-%d")
# use datetime to go back one week
print(start_date)

_debates_df = (
    debates_df[(debates_df["date"] >= start_date) & (debates_df["date"] <= end_date)]
    .merge(labelstore_df[["id", "mission_labels", "topic_labels"]], left_on="speech_id", right_on="id")
)

2024-10-03


In [148]:
import pandas as pd
people_df['persons'][-1]

# people_df['persons'][0]['other_names']

{'id': 'uk.org.publicwhip/person/26651',
 'identifiers': [{'identifier': '', 'scheme': 'pims_id'},
  {'identifier': '5371', 'scheme': 'datadotparl_id'}],
 'other_names': [{'county': '',
   'given_name': 'Deborah',
   'honorific_prefix': 'Bishop',
   'lordname': '',
   'lordofname': 'Peterborough',
   'lordofname_full': '',
   'note': 'Main',
   'surname': 'Sellin'}]}

In [154]:
pd.DataFrame(people_df['persons']).query("id == 'uk.org.publicwhip/person/26276'")

,id,identifiers,other_names,shortcuts,redirect
14215,uk.org.publicwhip/person/26276,"[{'identifier': '4998', 'scheme': 'datadotparl...","[{'family_name': 'Shanks', 'given_name': 'Mich...",NaN,NaN


In [155]:
pd.DataFrame(people_df['memberships']).query('person_id == "uk.org.publicwhip/person/26276"')

,end_date,end_reason,id,identifiers,person_id,post_id,start_date,start_reason,on_behalf_of_id,label,organization_id,role,redirect,reason,name
46470,2024-05-30,dissolution,uk.org.publicwhip/member/42754,NaN,uk.org.publicwhip/person/26276,uk.org.publicwhip/cons/714,2023-10-06,by_election,labour,NaN,NaN,NaN,NaN,NaN,NaN
46674,NaN,NaN,uk.org.publicwhip/member/42929,NaN,uk.org.publicwhip/person/26276,uk.org.publicwhip/cons/2739,2024-07-05,general_election,labour,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
orgs_df = pd.DataFrame(people_df['organizations'])[['id', 'name']]

In [162]:
# keep the most recent membership
parties_df = (
    pd.DataFrame(people_df['memberships'])
    .sort_values('start_date', ascending=False)
    .drop_duplicates('person_id')
    .merge(orgs_df, left_on='on_behalf_of_id', right_on='id', how='left', suffixes=("_", "_org"))
)[['person_id', 'post_id', 'start_date', 'start_reason', 'name_org']]
parties_df.query('person_id == "uk.org.publicwhip/person/11545"')

,person_id,post_id,start_date,start_reason,name_org
293,uk.org.publicwhip/person/11545,uk.org.publicwhip/cons/2423,2024-07-05,general_election,Labour


In [163]:
mission_debates = _debates_df.query("mission_labels == 'ASF'")
mission_debates = mission_debates.merge(parties_df[['person_id', 'name_org']], on = 'person_id', how='left', suffixes=('', '_person'))
debates_dates = mission_debates[['date', 'major_heading']].drop_duplicates().sort_values('date')

In [164]:
for _, row in debates_dates.iterrows():
    debate = row.major_heading
    print(debate)

Carbon Capture, Usage and Storage
Energy Security and Net Zero
Farming and Food Security
Prime Minister
Business of the House


In [175]:
row = debates_dates.iloc[4]
debate = row.major_heading
print("======")
print(debate)
print(row.date)
print("======")
speech = mission_debates.query("major_heading == @debate").to_dict(orient='records')
for s in speech:
    print(s['date'])
    print(f"{s['speakername']} ({s['name_org']})")
    print(f"Topics: {s['topic_labels']}")
    print(s['speech'])
    print("-----")

Business of the House
2024-10-10
2024-10-10
Chris Murray (Labour)
Topics: District heating
Around half a million people rely on heat networks, which is good in achieving our climate goals, but heat networks are not included in the Ofgem price cap. My constituents in Craigmillar are therefore facing a 400% increase in their energy bills. That is terrifying for them and completely unacceptable. Heat networks should have the same protections as traditional heating systems. Will the Leader of the House allow a debate in Government time so that we can address this terrible anomaly?
-----
2024-10-10
Lucy Powell (Labour/Co-operative)
Topics: Renewables - General
That very important matter has been raised with me before and I know that it is of real concern to my hon. Friend and to many other colleagues across this House. Next week’s Westminster Hall debate on the community benefits of renewable energy projects may provide him with an opportunity to raise it, but if not I will ensure that he g

In [135]:
for _, row in debates_dates.iterrows():
    debate = row.major_heading
    print("======")
    print(debate)
    print(row.date)
    print("======")
    speech = mission_debates.query("major_heading == @debate").to_dict(orient='records')
    for s in speech:
        print(s['date'])
        print(f"{s['speakername']} ({s['name_org']})")
        print(s['speech'])
        print("-----")

Carbon Capture, Usage and Storage
2024-10-07
2024-10-07
Bill Esterson (Labour)
Unlike the shadow Secretary of State, I am very pleased that the Secretary of State has announced jobs in Teesside—jobs from which my constituents in the north-west of England will potentially benefit. I am also very pleased that we have a Government who are committed to an industrial strategy, and who believe in Government working in partnership with business. The Secretary of State mentioned just how important it is that we have this technology if we are to decarbonise; he quoted James Richardson in making the case. It will be crucial for the abatement of heavy industries such as chemicals, glass—the Secretary of State went to visit a glass factory in the north-west on Friday—and cement, but it will also be crucial for hydrogen production, for the new gas-fired power stations and, indeed, for converting waste into energy. How long does he think we will need this technology for the abatement of heavy indust

In [23]:
speech

[{'speech_id': 'uk.org.publicwhip/debate/2024-10-07a.69.1',
  'speakername': 'Bill Esterson',
  'speaker_id': 'NA',
  'person_id': 'uk.org.publicwhip/person/24905',
  'speech': 'Unlike the shadow Secretary of State, I am very pleased that the Secretary of State has announced jobs in Teesside—jobs from which my constituents in the north-west of England will potentially benefit. I am also very pleased that we have a Government who are committed to an industrial strategy, and who believe in Government working in partnership with business. The Secretary of State mentioned just how important it is that we have this technology if we are to decarbonise; he quoted James Richardson in making the case. It will be crucial for the abatement of heavy industries such as chemicals, glass—the Secretary of State went to visit a glass factory in the north-west on Friday—and cement, but it will also be crucial for hydrogen production, for the new gas-fired power stations and, indeed, for converting waste